# Importing the data set

In [1]:
import os
import pandas as pd

test_path = ".\data\\test\\"
pos_path = ".\data\\train\\pos\\"
neg_path = ".\data\\train\\neg\\"

neg_review_list = []
pos_review_list = []
review_list = []
test_set = []

for file in os.listdir(neg_path):
    file_path = os.path.join(neg_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {}
    entry['class'] = 0
    entry['raw_txt'] = fh.read()
    neg_review_list.append(entry)
    fh.close()
    
for file in os.listdir(pos_path):
    file_path = os.path.join(pos_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {}
    entry['class'] = 1
    entry['raw_txt'] = fh.read()
    pos_review_list.append(entry)
    fh.close()

review_list = pd.DataFrame(neg_review_list[:20] + pos_review_list[:20])
review_list_test = pd.DataFrame(neg_review_list[20:30] + pos_review_list[20:30])
pos_review_list = pd.DataFrame(pos_review_list)
neg_review_list = pd.DataFrame(neg_review_list)


h


Preprocessing


In [9]:
import re
from nltk.corpus import stopwords
import numpy as np
stopWords = set(stopwords.words('english'))

def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['raw_txt'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
    
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    df['commas'] = df['raw_txt'].apply(lambda x: x.count(','))

    return df

df = processing(review_list)
print(df.head())

   class                                            raw_txt  \
0      0  Story of a man who has unnatural feelings for ...   
1      0  Airport '77 starts as a brand new luxury 747 p...   
2      0  This film lacked something I couldn't put my f...   
3      0  Sorry everyone,,, I know this is supposed to b...   
4      0  When I was little my parents took me along to ...   

                                           processed  length  words  \
0  story of a man who has unnatural feelings for ...     644    112   
1  airport 77 starts as a brand new luxury 747 pl...    4324    801   
2  this film lacked something i couldnt put my fi...     776    141   
3  sorry everyone i know this is supposed to be a...     832    154   
4  when i was little my parents took me along to ...    2265    395   

   words_not_stopword  avg_word_length  commas  
0                  63         6.365079       1  
1                 484         5.456612      16  
2                  64         6.375000       4 

In [14]:
from sklearn.model_selection import train_test_split
set = [c for c in df.columns.values if c  not in ['class']]
x_train, x_test, y_train, y_test = train_test_split(df[set],df['class'], test_size=0.20,random_state=10)
print(x_test.head())
print(y_test.head())

                                              raw_txt  \
2   This film lacked something I couldn't put my f...   
27  In this "critically acclaimed psychological th...   
35  Like one of the previous commenters said, this...   
30  When I first read Armistead Maupins story I wa...   
14  This movie must be in line for the most boring...   

                                            processed  length  words  \
2   this film lacked something i couldnt put my fi...     776    141   
27  in this critically acclaimed psychological thr...    2035    339   
35  like one of the previous commenters said this ...     852    158   
30  when i first read armistead maupins story i wa...    1511    280   
14  this movie must be in line for the most boring...     918    178   

    words_not_stopword  avg_word_length  commas  
2                   64         6.375000       4  
27                 221         6.153846      29  
35                  80         5.900000       5  
30                 151  

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator,TransformerMixin):
    def __init__(self, key):
        
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

In [ ]:
from sklearn.preprocessing import StandardScaler

length =  Pipeline([
                ('selector', NumberSelector(key='length')),
                ('standard', StandardScaler())
            ])

In [8]:
from sklearn.pipeline import FeatureUnion
features = FeatureUnion([('text', text),
                         
                         ])
features_process = Pipeline([('features', features)])


In [15]:
from sklearn.naive_bayes import BernoulliNB
pipeline = Pipeline([('features',features),
                     ('classifier', BernoulliNB())])
pipeline.fit(x_train,y_train)
prediction = pipeline.predict(x_test)
np.mean(prediction==y_test)

0.875